In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import urllib

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

In [5]:
from selenium.webdriver.chrome.options import Options

In [235]:
driver = webdriver.Chrome()

In [11]:
driver.get('https://www.airbnb.com/s/Tbilisi/homes')

In [30]:
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//*[@aria-label='Далее']"
        ))).click()

In [32]:
soup = BeautifulSoup(driver.page_source, 'html')
    
accomodation_list = soup.find_all('div', class_ = 'c4mnd7m atm_9s_11p5wf0 atm_dz_1osqo2v dir dir-ltr')
    

In [132]:
names = []
links = []
img = []
price = [] 
rating = []

#driver.get('https://www.airbnb.com/s/Tbilisi/homes')
#driver.maximize_window()

while True:
    
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html')
    
    accomodation_list = soup.find_all('div', class_ = 'c4mnd7m atm_9s_11p5wf0 atm_dz_1osqo2v dir dir-ltr')
    
    for i in accomodation_list:
        names.append(i.find('div', itemprop = 'itemListElement').find('meta', itemprop="name")['content'])
        links.append(i.find('div', itemprop = 'itemListElement').find('meta', itemprop="url")['content'].split('?')[0])
        if len(i.find_all('img')) > 1:
            img.append(i.find_all('img')[1]['src'])
        else:
            img.append(i.find('img')['src'])
        if len(
            i.find('div', class_ = '_w3xh25').find_all('span')) == 11:
            price_temp = i.find('div', class_ = '_w3xh25').find_all('span')[6].text
        else:
            price_temp = i.find('div', class_ = '_w3xh25').find_all('span')[2].text
        price.append(price_temp)
        try:
            rating.append(i.find('span', 
               class_ = 'r4a59j5 atm_h_1h6ojuz atm_9s_1txwivl atm_7l_jt7fhx atm_84_evh4rp atm_mk_h2mmj6 atm_mj_glywfm dir dir-ltr').find_all(
                'span')[2].text.split(' ')[0])
        except:
            rating.append('-')
        
    WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//*[@aria-label='Далее']"
        ))).click()

TimeoutException: Message: 


In [134]:
listings = pd.DataFrame({'names': names, 
                         'links': links,
                        'img': img,
                        'price': price,
                         'rating': rating})

In [136]:
listings.to_csv('listings_new.csv', index = False)

In [174]:
listings['ID'] = listings['links'].apply(
    lambda x: x.split('/')[len(x.split('/')) - 1])

In [151]:
detailed_listing = {}

In [153]:
listings = pd.read_csv('check.csv')

In [327]:
empty_reviews_keys = [key for key, value in detailed_listing.items() if not value.get('reviews')]
empty_reviews_keys

[1380114049428703488,
 1376931476865802752,
 1234839190842187520,
 1390693411979307008,
 1381669411314050816,
 976022503917158016,
 1380443960222775552,
 1315886132964956672,
 1358691672195489280,
 1378917923592876288,
 1254159379866241792,
 1317318078171343872,
 1378308370902653184,
 1377674172735145216,
 1312750008819000832,
 1381050138333080576]

In [333]:
listings = listings[listings['ID'].isin(empty_reviews_keys)].reset_index(drop=True)

In [337]:
for i in range(len(listings)):

    
    driver.get('https://' + listings['links'][i])

    time.sleep(1)
    
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH, "//*[@aria-label='Закрыть']"
                ))).click()
    except:
        pass
    
    soup = BeautifulSoup(driver.page_source, 'html')

    try:
        # Находим div с описанием
        description_block = driver.find_element(By.CSS_SELECTOR, 'div[data-section-id="DESCRIPTION_DEFAULT"]')
    
        # Ищем span с текстом внутри него
        description = description_block.find_element(By.CSS_SELECTOR, 'span').text
    except:
        description = ''

    properties = BeautifulSoup(driver.page_source, 'html').find_all('li',
                               class_ = 'l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr')

    guests = properties[0].text.replace('·', '').strip()
    bedrooms = properties[1].text.replace('·', '').strip()
    beds = properties[2].text.replace('·', '').strip()
    bathrooms = properties[3].text.replace('·', '').strip()
    
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, ".c1v4rb5q"
                ))).click()
        
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source, 'html')
    
        reviews = \
        [i.find(
            'span', class_ = 'l1h825yc atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr').text 
         for i in soup.find_all(
            'div', class_ = 'r1are2x1 atm_gq_1vi7ecw dir dir-ltr')]
    except:
        
        try:

            try:
                show_all_amenities = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(., 'Как устроена система отзывов')]"))
                    )
                    
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", show_all_amenities)
                
            except Exception as e:
                print("Кнопка 'Показать все удобства' не найдена:", e)

            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html')
            
            reviews = \
            [i.find(
                'span', class_ = 'l1h825yc atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr').text 
             for i in soup.find_all(
                'div', class_ = 'rjky5q atm_c8_16fp2vl atm_g3_x9fz81 atm_fr_12ckmjc atm_gq_1yuitx dir dir-ltr')]
        except:

            reviews = []

    # Закрыть модалку с отзывами, если открыта
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@aria-label='Закрыть']"))
        ).click()
    except:
        pass
    
    time.sleep(2)
    
    # Прокрутка до кнопки "Показать все удобства"
    try:
        try:
            show_all_amenities = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(., 'Показать все удобства')]"))
            )
            
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", show_all_amenities)
            time.sleep(1)
        
            # Кликаем через JavaScript (обходит overlay-проблему)
            driver.execute_script("arguments[0].click();", show_all_amenities)
        
        except Exception as e:
            print("Ошибка при попытке кликнуть на кнопку 'Показать все удобства':", e)
    except Exception as e:
        print("Кнопка 'Показать все удобства' не найдена:", e)
    
    # Собираем все удобства из открытого окна
    amenities = []
    try:
        # ждём, пока удобства появятся
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li div.f16mr5nx'))
        )
        
        # получаем все элементы с удобствами
        amenities_elements = driver.find_elements(By.CSS_SELECTOR, 'li div.f16mr5nx')
        amenities = [el.text.strip() for el in amenities_elements if el.text.strip()]
        
    except Exception as e:
        print("Ошибка при сборе удобств:", e)

    detailed_listing[listings['ID'][i]] = {
        'description': description,
        'reviews': reviews,
        'guests': guests,
        'bedrooms': bedrooms,
        'beds': beds,
        'bathrooms': bathrooms,
        'amenities': amenities
        }

    time.sleep(1)

In [341]:
test = detailed_listing.copy()

In [343]:
# convert list of dictionaries to a pandas DataFrame
test = pd.DataFrame(test)


In [345]:
test.columns = [str(x) for x in test.columns]

In [351]:
test

,709072965722063360,1388966597275632128,845715307056255872,1380114049428703488,50570531,1376931476865802752,24372957,1234839190842187520,39454860,1370448226296068608,...,41091976,1378308370902653184,5873903,1377674172735145216,38635868,1312750008819000832,17699001,1381050138333080576,41043054,1320121857714815488
description,,"Добро пожаловать в нашу скрытую жемчужину, в 2...","Небольшая и уютная квартира, идеально подходящ...","Добро пожаловать в нашу потрясающую квартиру, ...","Ваша семья будет рядом со всем, когда вы остан...",Вечеринки и шумные собрания не допускаются!\nК...,"Лучшее место, легко передвигаться во всех напр...",Сдается светлая и красивая квартира в новом до...,Наша стильная и светлая квартира 2BR с двумя в...,вы отлично проведете время в этом спокойном до...,...,"Elle 's Home расположен в старом Тбилиси, в са...",В 18 минутах ходьбы от Тбилисского театра опер...,Жилье\nнаша небольшая но очень уютная квартира...,"Люкс комфортабельный, недавно построенный, обо...","Недавно отремонтированная современная, комфорт...","Светлая, безупречная современная квартира всег...",Удобная и солнечная квартира с фантастическим ...,Расслабьтесь в этом спокойном жилье с потрясаю...,"КВАРТИРА расположена в Сабуртало, одном из цен...",Насладитесь тихой 1-комнатной квартирой всего ...
reviews,"[Очень уютная, чистая и стильная квартира с ши...","[Квартира великолепна, и в ней было все необхо...",[Очень хороший хозяин и очень довольны квартир...,"[Жилье находится на главной дороге, напротив о...","[Частная, уютная и комфортабельная база для зн...",[Номер был просторным и удобным. Супермаркет н...,[Потрясающая квартира! Мы останавливались с ма...,[Отличительная и чистая резиденция и очень тих...,"[Все было отлично. Жилье было безупречным, и в...","[Отлично провели время, в квартире есть все не...",...,[Жилье Ладо прекрасно расположено в центре гор...,"[Дом очень красивый, лучше, чем на фотографиях...","[Во-первых, было добро. Жилье было чистым, а с...","[👍, Останавливались в этой квартире на несколь...",[Мы прекрасно провели время — все основные дос...,"[Джеймс — лучший хозяин, и мне очень понравило...",[Мы провели несколько замечательных дней в ква...,[Очень хорошее жилье и много хороших услуг. Сп...,[Квартира была хорошо спроектирована и имела в...,"[Очень хорошее место — чистое, комфортабельное..."
guests,2 гостя,3 гостя,2 гостя,2 гостя,1 гость,2 гостя,6 гостей,2 гостя,4 гостя,3 гостя,...,2 гостя,2 гостя,2 гостя,4 гостя,3 гостя,4 гостя,4 гостя,3 гостя,2 гостя,2 гостя
bedrooms,1 спальня,1 спальня,1 спальня,1 спальня,1 спальня,1 спальня,2 спальни,1 спальня,2 спальни,1 спальня,...,1 спальня,1 спальня,1 спальня,1 спальня,студия,1 спальня,2 спальни,1 спальня,1 спальня,1 спальня
beds,1 кровать,1 кровать,1 кровать,1 кровать,1 кровать,1 кровать,2 кровати,2 кровати,2 кровати,1 кровать,...,1 кровать,1 кровать,2 кровати,2 кровати,1 кровать,2 кровати,2 кровати,3 кровати,1 кровать,1 кровать
bathrooms,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,2 ванные,1 ванная,...,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная,1 ванная
amenities,"[Вид на город, Ванна, Фен, Шампунь, Кондиционе...","[Сушильная машина в жилье: Бесплатно, Предметы...","[Плотные шторы или занавеси, Место для хранени...","[Постельное белье\nХлопковое постельное белье,...","[Фен, Стиральная машина, Сушильная машина, Тел...","[Стиральная машина в жилье: Бесплатно, Сушильн...","[Фен, Шампунь, Горячая вода, Стиральная машина...","[Утюг, Место для хранения одежды: гардеробная ...","[Шампунь, Горячая вода, Стиральная машина, Пре...","[Гель для душа, Стиральная машина, Предметы пе...",...,"[Место для хранения одежды: гардероб, Телевизо...","[Фен, Шампунь, Горячая вода, Постельное белье,...","[Шампунь, Горячая вода, Стиральная машина, Пре...","[Вид на горы, Кондиционер hotel, Гель для душа...","[Фен, Горячая вода, Стиральная машина, Предмет...","[Фен, Стиральная машина, Предметы первой необх...","[Горячая вода, Стиральная машина,

In [347]:
# convert DataFrame back to list of dictionaries
res = test.to_dict()

In [349]:
res

{'709072965722063360': {'description': '',
  'reviews': ['Очень уютная, чистая и стильная квартира с шикарным видом. Ощущаешь себя героем фильма, так как вокруг много предметов искусства и самого разного декора. Непременно советую всем, кто ищет жилье в Тбилиси и хочет провести время с наслаждением 🫶🏻',
   'Я абсолютно поражен этим лофтом. Каждая деталь\xa0— это как в музее, где хочется увидеть и прочитать все. Если вы хотите чего-то необычного, отправляйтесь сюда. И еще, да книге Коби! Я был приятно удивлен.',
   'ПОТРЯСАЮЩЕ',
   'Мы остановились в Chemia Studio на Пасху и в восторге😇Очень красиво меблированная квартира, которая убеждает своим характером. Вид из-за обеденного стола потрясающий и очаровывал нас каждое утро и вечер. Однажды ночью мы заметили, что у нас больше нет проточной воды, и написали Миха очень поздно. Он сразу сообщил нам, что это был разрыв трубы, и проблема была решена утром. С Миха очень легко связаться, и он очень хорошо информирован, что мы считали замечател

In [353]:
import json

# Пример: detailed_listing — это словарь
with open('detailed_listing_new2.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

In [432]:
with open('detailed_listing_new.json', 'r', encoding='utf-8') as f:
    detailed_listing_check = json.load(f)

In [434]:
detailed_listing_check

{'20411305': {'description': 'Уютная квартира в винтажном стиле расположена в Старом Тбилиси, на третьем этаже в Тбилисском дворе, недалеко от площади Свободы. Квартира находится в -8 минутах ходьбы от станций метро и внедорожников. Пешком до любой интересной части города можно дойти пешком, и она находится очень близко от нашего дома. Рядом есть все: театральные и оперные здания, старые церкви, музеи, кафе и рестораны.\n\n\nКвартира украшена старомодной мебелью, в квартире есть особая атмосфера, она создает ощущение прошлого, но в то же время очень современная. В нем есть чайник и место для хранения багажа. В квартире есть отдельная ванная комната с душем. Надеюсь, вам понравится.\n\n\nВся квартира в распоряжении гостей, и они могут пользоваться всеми удобствами в квартире.\n\n\nВремя прибытия и выезда гибкое. Пожалуйста, сообщите нам о вашем прибытии, чтобы мы могли договориться о времени.',
  'reviews': ['Очень хороший, очень со вкусом обставленный дом, было также очень приятно пров